In [1]:


import xarray as xr
import xesmf
#from HCtFlood.kara import flood_kara
#import bottleneck
import os
import numpy as np
import cftime
#%%


In [2]:
os.chdir("/home/nma/HDD_data/vinay/archives/IOMOM5Op/")
#exp_dir = '/home/nma/mom/vinay/MOM6dev/prepro/obc/'
exp_dir = '/home/nma/HDD_data/vinay/archives/IOMOM5Op/'

In [24]:
os.chdir("/home/nma/HDD_data/vinay/archives/IOMOM5Op/")

uu = xr.open_mfdataset("*.ocean_uvel.nc")
vv = xr.open_mfdataset("*.ocean_vvel.nc")
salt = xr.open_mfdataset("*.ocean_salt.nc")
ssh = xr.open_mfdataset("*.ocean_diag.nc").eta_t
temp = xr.open_mfdataset("*.ocean_temp.nc")
grd_p = "/home/nma/mom/MOM6dev/exps/2012_bob_obc/INPUT/horizontal_grid.nc"
#grd_p = "/home/nma/mom/MOM6dev/prepro/obc/obcmar/20120101.ocean_sfc.nc"
grid = xr.open_dataset(grd_p)
sam_op = xr.open_dataset("/home/nma/HDD_data/vinay/archives/finals/20130101.ocean_sfc.nc")

ds = xr.merge([uu,vv,salt,ssh,temp])
startslice = cftime.DatetimeNoLeap(2012, 1, 1,0,0,0,0,-1,1)
endslice = cftime.DatetimeNoLeap(2014, 1, 1,0,0,0,0,-1,1)

ds_cut = ds.sel(time=slice(startslice,endslice))

ds_cut
sam_op.yq.shape

(193,)

## Make Grid object

In [4]:


xgrid = grid.x[::2,::2]
ygrid = grid.y[::2,::2]

south = xr.Dataset()
#south['lon'] = grid['x'].isel(nyp=0)
#south['lat'] = grid['y'].isel(nyp=0)

south['lon'] = xgrid.isel(nyp=0)
south['lat'] = ygrid.isel(nyp=0)

#lats = np.ones(len(grid['xq'].values))*grid['yq'][0].values

#south['lon'] = grid['xq']
#south['lat'] = lats

south



<xarray.Dataset>
Dimensions:  (nxp: 301)
Dimensions without coordinates: nxp
Data variables:
    lon      (nxp) float64 ...
    lat      (nxp) float64 ...

In [5]:


regrid_south = xesmf.Regridder(ds_cut.rename({'xu_ocean': 'lon', 'yu_ocean': 'lat'}), south, 'bilinear', 
                               locstream_out=True, periodic=False, filename='regrid_south.nc')



In [6]:
ssh_south

NameError: name 'ssh_south' is not defined

In [7]:
v_south = regrid_south(ds_cut['v'])
u_south = regrid_south(ds_cut['u'])
temp_south = regrid_south(ds_cut['temp'])
ssh_south = regrid_south(ds_cut['eta_t'])
salt_south = regrid_south(ds_cut['salt'])

/home/nma/nma/miniconda3/envs/mom/lib/python3.8/site-packages/xesmf/frontend.py:594: UserWarning: Using dimensions ('yu_ocean', 'xu_ocean') from data variable v as the horizontal dimensions for the regridding.
  warnings.warn(
/home/nma/nma/miniconda3/envs/mom/lib/python3.8/site-packages/xesmf/frontend.py:594: UserWarning: Using dimensions ('yu_ocean', 'xu_ocean') from data variable u as the horizontal dimensions for the regridding.
  warnings.warn(
/home/nma/nma/miniconda3/envs/mom/lib/python3.8/site-packages/xesmf/frontend.py:594: UserWarning: Using dimensions ('yt_ocean', 'xt_ocean') from data variable temp as the horizontal dimensions for the regridding.
  warnings.warn(
/home/nma/nma/miniconda3/envs/mom/lib/python3.8/site-packages/xesmf/frontend.py:594: UserWarning: Using dimensions ('yt_ocean', 'xt_ocean') from data variable eta_t as the horizontal dimensions for the regridding.
  warnings.warn(
/home/nma/nma/miniconda3/envs/mom/lib/python3.8/site-packages/xesmf/frontend.py:594: 

<xarray.DataArray (time: 730, nxp: 301)>
dask.array<getitem, shape=(730, 301), dtype=float32, chunksize=(515, 301), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2012-01-01 12:00:00 ... 2013-12-31 12:00:00
    lon      (nxp) float64 77.0 77.07 77.15 77.22 ... 98.78 98.85 98.93 99.0
    lat      (nxp) float64 4.0 4.0 4.0 4.0 4.0 4.0 ... 4.0 4.0 4.0 4.0 4.0 4.0
Dimensions without coordinates: nxp
Attributes:
    regrid_method:  bilinear

In [8]:
drowned_v_south = v_south.ffill(dim='nxp').ffill(dim='st_ocean')
drowned_u_south =u_south.ffill(dim='nxp').ffill(dim='st_ocean')
drowned_temp_south = temp_south.ffill(dim='nxp').ffill(dim='st_ocean')
drowned_salt_south = salt_south.ffill(dim='nxp').ffill(dim='st_ocean')
drowned_ssh_south = ssh_south.ffill(dim='nxp')

In [25]:
n_southv_array = np.expand_dims(drowned_v_south.values,axis=2)
n_southu_array = np.expand_dims(drowned_u_south.values,axis=2)
n_southtemp_array = np.expand_dims(drowned_temp_south.values,axis=2)
n_southsalt_array = np.expand_dims(drowned_temp_south.values,axis=2)


In [10]:
n_southssh_array = np.expand_dims(ssh_south.values,axis=2)
n_southssh_array.shape

(730, 301, 1)

In [16]:
drowned_u_south

<xarray.DataArray (time: 730, st_ocean: 58, nxp: 301)>
dask.array<push, shape=(730, 58, 301), dtype=float32, chunksize=(515, 58, 301), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 0.5 1.5 2.5 3.5 ... 3.625e+03 4.3e+03 5e+03
  * time      (time) object 2012-01-01 12:00:00 ... 2013-12-31 12:00:00
    lon       (nxp) float64 77.0 77.07 77.15 77.22 ... 98.78 98.85 98.93 99.0
    lat       (nxp) float64 4.0 4.0 4.0 4.0 4.0 4.0 ... 4.0 4.0 4.0 4.0 4.0 4.0
Dimensions without coordinates: nxp
Attributes:
    regrid_method:  bilinear

In [27]:

south_final = xr.Dataset({
    "v" : (["time","zl","yq","xh"],n_southv_array),
"u" : (["time","zl","yq","xh"],n_southu_array),
"temp" : (["time","zl","yq","xh"],n_southtemp_array),
"salt" : (["time","zl","yq","xh"],n_southsalt_array),
"ssh" : (["time","xh","yq"],n_southssh_array)},
    coords = {"time":(["time",],drowned_v_south.time.values),
              "zl":(["zl",],drowned_v_south.st_ocean.values),
              "yq":(["yq",],[drowned_v_south.lat.values[0]]),
              "xh":(["xh"],drowned_v_south.lon.values)})

In [28]:
south_final

<xarray.Dataset>
Dimensions:  (time: 730, zl: 58, yq: 1, xh: 301)
Coordinates:
  * time     (time) object 2012-01-01 12:00:00 ... 2013-12-31 12:00:00
  * zl       (zl) float64 0.5 1.5 2.5 3.5 4.5 ... 3e+03 3.625e+03 4.3e+03 5e+03
  * yq       (yq) float64 4.0
  * xh       (xh) float64 77.0 77.07 77.15 77.22 ... 98.78 98.85 98.93 99.0
Data variables:
    v        (time, zl, yq, xh) float32 -0.2013 -0.195 ... 0.04189 0.04189
    u        (time, zl, yq, xh) float32 -0.7064 -0.6969 ... 0.002261 0.002261
    temp     (time, zl, yq, xh) float32 27.87 27.86 27.88 ... 1.413 1.413 1.413
    salt     (time, zl, yq, xh) float32 27.87 27.86 27.88 ... 1.413 1.413 1.413
    ssh      (time, xh, yq) float32 -0.03282 -0.03379 -0.03473 ... nan nan nan

In [29]:
caops_dir = "/home/nma/HDD_data/vinay/archives/IOMOM5Op/caops/"
sam_obc = xr.open_dataset(caops_dir+"obc_east.nc")
dz = sam_obc.dz_u_segment_002
dirv = "/home/nma/mom/MOM6dev/prepro/obc/gen_vgrid/"
IC_grid = xr.open_dataset(dirv+"2013_final.nc")
hy_depth = IC_grid.DEPTH.values
sam_obc

<xarray.Dataset>
Dimensions:             (xh: 1, yh: 192, xq: 1, yq: 193, zl: 40, time: 34)
Coordinates:
  * xh                  (xh) float64 -77.04
  * yh                  (yh) float64 18.09 18.17 18.24 ... 31.76 31.82 31.89
  * xq                  (xq) float64 -77.08
  * yq                  (yq) float64 18.05 18.13 18.21 ... 31.79 31.86 31.93
  * zl                  (zl) float64 40.0 9.969e+36 ... 9.969e+36 9.969e+36
  * time                (time) datetime64[ns] 2014-12-31T12:00:00 ... 2015-02...
Data variables: (12/13)
    u_segment_002       (time, zl, yh, xq) float64 ...
    v_segment_002       (time, zl, yq, xh) float64 ...
    ssh_segment_002     (time, yh, xh) float64 ...
    dz_u_segment_002    (time, zl, yh, xq) float64 ...
    dz_v_segment_002    (time, zl, yq, xh) float64 ...
    pt_segment_002      (time, zl, yh, xh) float64 ...
    ...                  ...
    dz_pt_segment_002   (time, zl, yh, xh) float64 ...
    dz_s_segment_002    (time, zl, yh, xh) float64 ...
    u11_segment_002     (time, zl, yh, xq) float64 ...
    v11_segment_002     (time, zl, yq, xh) float64 ...
    dz_u11_segment_002  (time, zl, yh, xq) float64 ...
    dz_v11_segment_002  (time, zl, yq, xh) float64 ...

In [31]:
south_obc = south_final
cur_depth = south_obc.zl.values

reg_south_obc = south_obc.interp(zl=hy_depth)


south_dz_new = np.array(len(reg_south_obc.xh.values)*[np.array(len(reg_south_obc.yq.values)*[np.array(len(reg_south_obc.time.values)*[np.insert(np.diff(hy_depth),
                                                                                      0,1)])])])


final_dz = south_dz_new.transpose(2,3,1,0)

fin_reg_south = reg_south_obc.assign({"dz_v":(["time","zl","yq","xh"],final_dz),
                                     "dz_temp" : (["time","zl","yq","xh"],final_dz),
                                     "dz_salt" : (["time","zl","yq","xh"],final_dz),
                                     "dz_u" : (["time","zl","yq","xh"],final_dz)})

fin_reg_south.to_netcdf("/home/nma/mom/MOM6dev/prepro/obc/obcflather/nsouth.nc")